# Cloud Run Test Env

# Cloud Run Test Env Simple App

In [ ]:
# Simple App

In [ ]:
!mkdir cloud-run-test-env

In [ ]:
cd cloud-run-test-env

In [ ]:
%%writefile env.py
NAME="Matt"

In [ ]:
%%writefile app.py
import streamlit as st
from env import *
import os

st.write("It's working.")
st.write(NAME)
st.write(os.getenv("NAME"))
st.write(os.environ.get("NAME"))

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit -q
COPY . /app
EXPOSE 9000
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0", "--server.port", "9000"]

In [ ]:
!docker build -t simple-app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 9000:9000 -v $(pwd):/app --name simple-app simple-app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
!docker rm -f simple-app

#### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:8501,tcp:9000 --source-ranges=0.0.0.0/0

### Deployment

#### Deploy using Cloud Build, Artifact Registry, and Cloud Run

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create simple-app \
    --repository-format=docker \
    --location=us-west2 \
    --description="Docker repository"

In [ ]:
%%sh
# Verify
gcloud artifacts repositories list

#### Build an image using Dockerfile

In [ ]:
# Check if the path is correct
!pwd

In [ ]:
cd cloud-run-test-env

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --tag us-west2-docker.pkg.dev/$(gcloud config get-value project)/simple-app/simple-app:tag1

In [ ]:
# It's working.